# Predicting Earnings Surprises

## Task

We want to predict the magnitude of company's upcoming earnings announcement using a machine learning classification model. The model is trained on three types of data: earnings, pricing, and technical price action data. The optimized model outputs a result into one of three classes: positive, neutral, or negative. A 'positive' classification indicates a predicted surprise >15% of the estimated eps, a 'negative' classification indicates a predicted surprise <-15% of the estimated eps, and a 'neutral' classification indicates no predicted surprise (15% < x < -15%). 

## Data

The data for training and testing the model came from several external data providers. Earning and pricing data was collected from Financial Modeling Prep's historical earnings calendar and daily indicator endpoints. Technical data is collected from FMP Cloud's daily technical indicator endpoint.

The schema below outlines the database architecture into an AWS RDS MySQL database:

![Untitled Workspace (1)](https://user-images.githubusercontent.com/45079557/150410944-eb8c8e30-ac2d-4f23-bb03-cb5c3f489cfb.png)

In [1]:
import pandas as pd
import numpy as np
import pymysql
from decouple import config
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
# Set parameters for AWS database
aws_hostname = config("AWS_HOST")
aws_database = config("AWS_DB")
aws_username = config("AWS_USER")
aws_password = config("AWS_PASS")
aws_port = config("AWS_PORT")

# Pull API keys from .env file
FMP_API_KEY = config("FMP_API_KEY")
FMP_CLOUD_API_KEY = config("FMP_CLOUD_API_KEY")

In [3]:
db = pymysql.connect(host=aws_hostname,user=aws_username, password=aws_password, database='rds-python', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()

In [4]:
cursor.execute("SELECT e.*, p.*, t.* FROM earnings e INNER JOIN pricing p ON e.id = p.id INNER JOIN technicals t ON e.id = t.id")
train = cursor.fetchall()

In [10]:
train_df = pd.DataFrame(train)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75781 entries, 0 to 75780
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 75781 non-null  object 
 1   earnings_date      75781 non-null  object 
 2   symbol             75781 non-null  object 
 3   eps                75578 non-null  float64
 4   eps_estimated      75781 non-null  float64
 5   earnings_time      75781 non-null  object 
 6   p.id               75781 non-null  object 
 7   p.earnings_date    75781 non-null  object 
 8   p.symbol           75781 non-null  object 
 9   open_price         75781 non-null  float64
 10  high_price         75781 non-null  float64
 11  low_price          75781 non-null  float64
 12  close_price        75781 non-null  float64
 13  adj_close          75781 non-null  float64
 14  daily_volume       75781 non-null  int64  
 15  unadjusted_volume  75781 non-null  int64  
 16  change_dollars     757

In [11]:
cursor.close()
db.close()

In [ ]:
train_df.drop(['p.id', 'p.earnings_date', 'p.symbol', 't.id', 't.earnings_date', 't.symbol'], axis = 1)

In [ ]:

is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]